In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('./austen_listings.csv')

C:\Users\PhatDeluxe\Anaconda3\envs\tensor_flow\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('max_columns', None)
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2265,https://www.airbnb.com/rooms/2265,20191112224534,2019-11-13,Zen-East in the Heart of Austin (monthly rental),Zen East is situated in a vibrant & diverse mu...,This colorful and clean 1923 house was complet...,Zen East is situated in a vibrant & diverse mu...,none,NaN,A 2013 Genuine Buddy Scooter 125 may be availa...,5 min walk to Capitol Metro Rail (train that t...,"Several local restaurants, small clubs, music ...","Depending on your dates and arrival time, I am...",• Check-in time is 4 pm. Check out is 11 am. I...,NaN,NaN,https://a0.muscache.com/im/pictures/4740524/63...,NaN,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,NaN,t,https://a0.muscache.com/im/users/2466/profile_...,https://a0.muscache.com/im/users/2466/profile_...,East Downtown,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"Austin, TX, United States",East Downtown,78702,NaN,Austin,TX,78702,Austin,"Austin, TX",US,United States,30.27750,-97.71398,f,House,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$225.00,NaN,NaN,$500.00,$100.00,4,$30.00,30,90,30,30,90,90,30.0,90.0,3 months ago,t,0,0,0,0,2019-11-13,24,1,2009-03-17,2019-03-16,93.0,9.0,10.0,10.0,10.0,8.0,9.0,f,NaN,"{""Texas State""}",f,f,strict_14_with_grace_period,f,f,3,2,1,0,0.18
1,5245,https://www.airbnb.com/rooms/5245,20191112224534,2019-11-13,"Eco friendly, Colorful, Clean, Cozy monthly share",Situated in a vibrant & diverse multicultural ...,"This green, colorful, clean and cozy house was...",Situated in a vibrant & diverse multicultural ...,none,NaN,Please note: A two story studio was built in t...,NaN,NaN,"I should be available, upon your arrival, to a...",A brief profile for all guests along with phot...,NaN,NaN,https://a0.muscache.com/im/pictures/5167505/b3...,NaN,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,NaN,t,https://a0.muscache.com/im/users/2466/profile_...,https://a0.muscache.com/im/users/2466/profile_...,East Downtown,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"Aust

In [4]:
df.shape

(11250, 106)

In [ ]:
for column in df.columns:
    print(column)

In [6]:
df['price'].value_counts()

$150.00      381
$100.00      360
$200.00      318
$250.00      262
$125.00      255
            ... 
$352.00        1
$1,925.00      1
$6,798.00      1
$1,280.00      1
$1,225.00      1
Name: price, Length: 682, dtype: int64

In [34]:
df_copy = pd.DataFrame()

In [35]:
def price_to_float(price):
    price = price.replace('$', '')
    price = price.replace(',', '')
    float(price)
    return price

df_copy['price'] = df['price'].apply(string_to_int)

In [36]:
df_copy.head()

,price
0,225.00
1,100.00
2,95.00
3,40.00
4,99.00


In [48]:
def zipcode_to_int(zipcode):
    zipcode = str(zipcode)
    zipcode = zipcode.replace('TX', '')
    zipcode = zipcode.replace(' ', '')
    zipcode = zipcode.replace('.0', '')
    zipcode = int(zipcode)
    return zipcode

df_copy['zipcode'] = df['zipcode'].apply(zipcode_to_int)

In [49]:
df['zipcode'] = df_copy['zipcode']

In [8]:
len(df.columns)

106

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [19]:
features = ['bedrooms', 'bathrooms', 'zipcode']
target = 'price'

In [20]:
df[features].isnull().sum()

bedrooms     0
bathrooms    0
zipcode      0
dtype: int64

In [17]:
df[features] = df[features].fillna(method='ffill')

In [25]:
model = Sequential()

model.add(Dense(3, input_dim=3, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1))

model.compile(loss='MSE', optimizer='adam', metrics=['mean_squared_error'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 3         
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.fit(df[features], df_copy['price'], epochs=5000, verbose=1, validation_split=1)

Train on 11250 samples
Epoch 1/150
11250/11250 [==============================] - 1s 57us/sample - loss: 1825227.3445 - mean_squared_error: 1825227.8750
Epoch 2/150
11250/11250 [==============================] - 1s 62us/sample - loss: 1824997.2687 - mean_squared_error: 1824997.3750
Epoch 3/150
11250/11250 [==============================] - 1s 68us/sample - loss: 1824767.8805 - mean_squared_error: 1824768.7500
Epoch 4/150
11250/11250 [==============================] - 1s 65us/sample - loss: 1824537.4396 - mean_squared_error: 1824537.8750
Epoch 5/150
11250/11250 [==============================] - 1s 58us/sample - loss: 1824307.4806 - mean_squared_error: 1824307.6250
Epoch 6/150
11250/11250 [==============================] - 1s 62us/sample - loss: 1824078.5565 - mean_squared_error: 1824079.1250
Epoch 7/150
11250/11250 [==============================] - 1s 60us/sample - loss: 1823849.8337 - mean_squared_error: 1823849.2500
Epoch 8/150
11250/11250 [==============================] - 1s 58us/